In [1]:
import tensorflow as tf

In [2]:
import pandas as pd

In [3]:
iris_data = pd.read_csv('Iris.csv')

In [4]:
iris_data.head() #prints the first 5 rows

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [7]:
print(iris_data.shape)

(150, 6)


In [8]:
iris_data_one_hot_encoded = pd.get_dummies(iris_data)

In [9]:
iris_data_one_hot_encoded.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


In [10]:
print(iris_data_one_hot_encoded.shape)

(150, 8)


In [13]:
#80% Training - 20% Testing
iris_test_data = iris_data_one_hot_encoded.sample(frac=0.2, random_state=200)
iris_train_data = iris_data_one_hot_encoded.drop(iris_test_data.index)

In [14]:
iris_test_data.index

Int64Index([ 84, 122,  28,  24,  75, 109,  81,  98,  80, 100, 124,   2,  34,
             44, 128,  13,  93,  41,  63, 137,  19,  60, 126,  11,  47,  18,
             96, 107, 141,  53],
           dtype='int64')

In [15]:
iris_train_input_data = iris_train_data.filter(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'] )
iris_train_label_data = iris_train_data.filter(['Species_Iris-setosa', 'Species_Iris-versicolor','Species_Iris-virginica'])

In [16]:
iris_test_input_data = iris_test_data.filter(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'] )
iris_test_label_data = iris_test_data.filter(['Species_Iris-setosa', 'Species_Iris-versicolor','Species_Iris-virginica'])

In [19]:
# y = w * x + b
input_features = tf.placeholder(tf.float32, [None, 4])
W = tf.Variable(tf.zeros([4,3]))

#input_data = [5.5 , 2.3, 4.0, 1.3]
#W = [[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
b = tf.Variable(tf.zeros([3]))

predicted_label = tf.nn.softmax(tf.matmul(input_features, W) + b)
actual_label = tf.placeholder(tf.float32, [None,3])

In [20]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(actual_label \
        * tf.log(predicted_label), reduction_indices=[1]))

In [21]:
lr = 0.05
train_step = tf.train.GradientDescentOptimizer(lr).minimize(cross_entropy)

In [22]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for step in range(1000):
    sess.run(train_step, 
             feed_dict={
                 input_features: iris_train_input_data,
                 actual_label: iris_train_label_data
                })

In [24]:
#predicted_label = [0.06, 0.9, 0.04] -> [0,1,0]

# setosa = [1,0,0]
# versicolor = [0,1,0]
# virginica = [0,0,1]

# actual_label = [0,1,0] versicolor

# 1. get the index of the highest number in the predicted
## 1
# 2. get the index of the highest number in the actual label
## 1
# 3. compare the index
## 1 == 1 = TRUE!

correct_prediction = tf.equal(tf.argmax(predicted_label, 1),
                              tf.argmax(actual_label, 1))

# [1,1,1,1,1,1,0,1,1,0] = 8
# 8 / 10 = (0.8)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [25]:
computed_accuracy = sess.run(accuracy, feed_dict={
                 input_features: iris_test_input_data,
                 actual_label: iris_test_label_data
                })

print("Accuracy: ", computed_accuracy)

Accuracy:  0.966667
